In [97]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [98]:
import torch
from torch import nn as nn
from torch.utils.data import Dataset
from torch import optim
from torch.nn import functional as F
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

DEVICE = torch.device('cpu')

In [99]:
pd.read_csv('diabetes.csv')

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [100]:
#cream clasa dataset care o sa ne transforme tabelul in tensor
class Dataset(Dataset):
    #facem un constructor de initiere
    def __init__(self, path):
        #citim valorile din fisier
        self.dataset = pd.read_csv(path).values
        
    def __len__(self):
        #returnam lungimea la dataset
        return len(self.dataset)
    
    #extragem un element din date
    def __getitem_(self, i):
        #returnam datele x si y cu indexul i
        return self.dataset[i,:-1], self.dataset[i,-1]

In [101]:
# structurarea retelei neuronale
class Neural_Network(nn.Module):
    #facem o functie si specificam citi neuroni o sa fie la intrare
    def __init__(self, n_features):
        #chemam constructorul din clasa parinte ca sa intializam reteaua neuronala
        super().init__init__()
        #setam nr de valori initiale
        #care o sa fie egal cu nr de features din dataset
        self.n_features = n_features
        
        #Cream straturi pentru arhitectura noastra
        self.fc1 = nn.Linear(self.n_features, 8) # (input, ouput)     
        self.fc2 = nn.Linear(8, 32) # (input, ouput)     
        self.fc3 = nn.Linear(32, 4) # (input, ouput)  
        
    #cream functia forward, x trebuie sa  fie de lungimea n_features
    def forward(self, x):
        #out reprezinta varaibila care va tine raspunsul dupa un anumit strat
        #F.relu - functie de activare (intre straturi)
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        out = F.sigmoid(self.fc3(out))
        
        #returning ouput
        return out
        

In [102]:
# cream ciclul de antrenare
def Ciclu_de_antrenare(n_epochs, optimizer, model, loss_fn, train_loader, test_loader, print_plot=True):

#Inițializarea a două liste pentru a salva acuratețea pentru a le reprezenta mai târziu   
    train_accuracy = torch.zeros(n_epochs)
    test_accuracy = torch.zeros(n_epochs)
    
    #cream si variabile pentru pierdere (train si loss)
    train_loss = np.zeros(n_epochs)
    test_loss = np.zeros(n_epochs)
    
    #Ne ducem prin fiecare epoca
    for epoch in range(n_epochs):

        # schimbam switchul si il punem pe train
        model.train()
        total = 0
        correct = 0
        current_train_loss = 0.0
        
        #Mergem prin exemplu, prin fiecare lot, în cazul nostru, prin toate datele simultan
        for example, labels in train_loader:     
            
            #Traducerea calculelor în GPU dacă este disponibil
            #toate calcularile vor fi pe acelasi procesor
            example = example.to(DEVICE).float()
            labels = labels.to(DEVICE).float()

            #adaugam o dimensiune
            labels  = labels.unsqueeze(1)

            # ii dam modelul de exemple
            predicted = model(example)
            
            # Pierderea rețelei 
            loss = loss_fn(predicted, labels)
            
            #tinem cont care este erroarea curenta
            current_train_loss += loss
            
            #aflam cite exemple sunt in total
            total += labels.shape[0]
            
            #rotungim datele
            predicted = torch.round(predicted)
            
            #aflam cite sunt prezise corect
            correct += (predicted == labels).sum()
            
            # Reducerea la zero a gradientului pentru a nu-l stivui din alte iterații
            optimizer.zero_grad()
            
            #Alergarea în spate a rețelei neuronale, obținând pante
            loss.backward()
            
            #facem update la parametri
            optimizer.step()
         
        #Calculam acuratetea, le vom folosi  ca sa facem grafice
        train_accuracy[epoch] = correct / total
        train_loss[epoch] = current_train_loss / total
        
        
        # Testam si afisam
        model.eval()
        correct = 0
        total = 0
        current_test_loss = 0
        
            
        correct = 0
        total = 0
         #Mergem prin exemplu, prin fiecare lot,
        for examples, labels in test_loader:
            examples = examples.to(device=DEVICE).float()
            labels = labels.to(device=DEVICE).float()
            
            labels  = labels.unsqueeze(1)

            predicted = model(examples)
            
            loss = loss_fn(predicted, labels)
            current_test_loss += loss
            
            total += labels.shape[0]
            predicted = torch.round(predicted)

            correct += (predicted  == labels).sum()

        test_accuracy[epoch] = correct / total
        test_loss[epoch] = current_test_loss / total
        
        #verificam daca ultima cifra este 0
        if (epoch+1) % 10 ==0:
            #afisam de la 1
            print(f'Epoch {epoch+1})', 
                  f'Train Accuracy: {train_accuracy[epoch]}',
                  f'Test Accuracy: {test_accuracy[epoch]}')
            
    #Dacă este setat la True, imprimați graficul trenului și pierderea de validare
    if print_plot:

        #setam axa de epoci
        epochs_range = range(1,n_epochs+1)
        
       # facem 2 grafice
        plt.subplots(figsize=(10, 6))
        
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, train_loss, 'g', label='Training loss')
        plt.plot(epochs_range, test_loss, 'b', label='Test loss')
        plt.title('Training and Test Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        #Trasarea ambelor curbe, tren și val
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, train_accuracy, 'g', label='Training accuracy')
        plt.plot(epochs_range, test_accuracy, 'b', label='Test accuracy')
        plt.title('Training and Test Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

In [103]:
#Crearea unui set de date din fișierul csv
data = Dataset(r'C:\Users\acer\SQL SIGMOID\diabetes.csv')

n_samples = len(data)
#pentru testare se ia 20% si o transformam in cifra intreaga
n_test = int(0.2*n_samples)

#Salvam nr de exemple și calcularea numărului de probe de validare
train_set, test_set = torch.utils.data.random_split(data,  [n_samples-n_test, n_test])

# rulam trenul și seturile de validare prin încărcătorul de date care ajută la paralelizarea procesului de încărcare a datelor cu loturi automate
train_loader = torch.utils.data.DataLoader(train_set, batch_size=len(train_set), shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=len(test_set), shuffle=True)


In [104]:
#Setarea unor hiperparametri și parametri
learning_rate = 1e-2

#Inițializarea modelului cu nr de caracteristici din intrare
model = Net(len(data[0][0])).to(DEVICE)

#Facem optimizerul
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
#calculeaza eroare..
loss_fn = nn.BCELoss()                                                     

#Dam parametrii de intrare a functiei
Ciclu_de_antrenare(
    n_epochs=500,
    optimizer=optimizer,
    model = model,
    loss_fn = loss_fn,
    print_plot=True,
    train_loader=train_loader,
    test_loader = test_loader
)


#Nu se afiseaza graficul , nu merge Net()

NameError: name 'Net' is not defined